In [1]:
import pandas as pd 

In [8]:
people = pd.read_csv("EHRShot/sampled_person.csv")
deaths = pd.read_csv("EHRShot/sampled_death.csv")
sites = pd.read_csv("EHRShot/care_site.csv")
conditions = pd.read_csv("EHRShot/sampled_condition_occurrence.csv")

C:\Users\patri\AppData\Local\Temp\ipykernel_18664\1278085068.py:4: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  conditions = pd.read_csv("EHRShot/sampled_condition_occurrence.csv")


In [ ]:
# use regex to match ICU/"intensive care"
pattern = r'\bICU\b|intensive care'
mask = sites['care_site_name'].fillna('').str.contains(pattern, case=False, regex=True)
icus = sites.loc[mask].copy()

icus.head()

,care_site_id,care_site_name,place_of_service_concept_id,location_id,care_site_source_value,place_of_service_source_value,trace_id,unit_id,load_table_id
70,529546,ICU,8756,8646458,110100006,On Campus - Outpatient Hospital,NaN,NaN,shc_clarity_dep
71,529617,INTENSIVE CARE,8756,11352401,20026201,On Campus - Outpatient Hospital,NaN,NaN,shc_clarity_dep
213,529578,ICU,8756,10178563,110100002,On Campus - Outpatient Hospital,NaN,NaN,shc_clarity_dep
451,527995,PEDIATRIC INTENSIVE CARE,0,11101381,9991085,NaN,NaN,NaN,shc_clarity_dep
508,527283,INTENSIVE CARE,0,10537544,10201253,NaN,NaN,NaN,lpch_clarity_dep


In [12]:
# find persons whose care_site is in the ICUs list
#! not working
mask = people['care_site_id'].isin(set(icus['care_site_id']))
people_in_icus = people.loc[mask].copy()

print(f'People total: {len(people)}, People in ICUs: {mask.sum()}')

People total: 2000, People in ICUs: 0


In [13]:
# find conditions where the person has an associated record in deaths
mask = conditions['person_id'].isin(set(deaths['person_id']))
conditions_with_deaths = conditions.loc[mask].copy()

print(f'Conditions total: {len(conditions)}, Conditions with deaths: {mask.sum()}')
print(f'People total: {len(people)}, People suffering death: {len(set(conditions_with_deaths["person_id"]))}')

Conditions total: 761822, Conditions with deaths: 175475
People total: 2000, People suffering death: 207
